# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.2 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: 93b50815-5204-4657-8ad8-2a9fdb36f567
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
Waiting for session 93b50815-5204-4657-8ad8-2a9fdb36f567 to get into ready status...
Session 93b50815-5204-4657-8ad8-2a9fdb36f567 has been created.



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='proyectoint20232', table_name='file_csv')
dyf.printSchema()

root
|-- col1: string
|-- col2: string
|-- col3: string
|-- col4: string
|-- col5: string
|-- col6: string
|-- col7: string
|-- col8: string
|-- col9: string
|-- col10: string
|-- col11: string
|-- col12: string
|-- col13: string
|-- col14: string
|-- col15: string
|-- col16: string
|-- col17: string
|-- col18: string
|-- col0: long


In [3]:
# Definir la lista de los nuevos nombres de las columnas
column_names = [
    "serial", "mfr", "fecha", "requested_test_kv", "test_kv", "ma", "watts",
    "measured_cap", "pfm", "pfm_tipup", "line_id", "group_number", "circuit",
    "location", "division", "company", "cct_designation", "stator_kv"
]

# Renombrar las columnas en el DynamicFrame
for i in range(1, 19):  # Desde col1 hasta col18
    old_column_name = f"col{i}"
    new_column_name = column_names[i - 1]
    dyf = dyf.rename_field(old_column_name, new_column_name)

# Verificar el esquema después de renombrar
dyf.printSchema()

root
|-- serial: string
|-- mfr: string
|-- fecha: string
|-- requested_test_kv: string
|-- test_kv: string
|-- ma: string
|-- watts: string
|-- measured_cap: string
|-- pfm: string
|-- pfm_tipup: string
|-- line_id: string
|-- group_number: string
|-- circuit: string
|-- location: string
|-- division: string
|-- company: string
|-- cct_designation: string
|-- stator_kv: string
|-- col0: long


In [4]:
from awsglue.transforms import *

# Columnas a eliminar
columns_to_drop = ["line_id", "circuit", "location", "division", "company", "cct_designation","col0"]

# Utilizar DropFields para eliminar las columnas
dyf = DropFields.apply(frame=dyf, paths=columns_to_drop)

# Verificar el esquema después de la eliminación
dyf.printSchema()

root
|-- serial: string
|-- mfr: string
|-- fecha: string
|-- requested_test_kv: string
|-- test_kv: string
|-- ma: string
|-- watts: string
|-- measured_cap: string
|-- pfm: string
|-- pfm_tipup: string
|-- group_number: string
|-- stator_kv: string


#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [5]:
DF = dyf.toDF()
DF.show()

+------+-------+-------------------+-----------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------+---------+
|serial|    mfr|              fecha|requested_test_kv|           test_kv|                ma|              watts|      measured_cap|                pfm|          pfm_tipup|group_number|stator_kv|
+------+-------+-------------------+-----------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------+---------+
|serial|    mfr|              fecha|requested_test_kv|           test_kv|                ma|              watts|      measured_cap|                pfm|          pfm_tipup|group_number|stator_kv|
|715015|Toshiba|2018-11-27 16:07:37|              2.0|1.9947197437286377|3572.8927850723267|  388.1911926269531| 948070.8058617892|  1.086489900421389|                   |           0|     13.8|
|715015|Toshiba|2018-11-2

In [6]:
# Convertir el DataFrame de Spark en un DataFrame de pandas
df = DF.toPandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17355 entries, 0 to 17354
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   serial             17355 non-null  object
 1   mfr                17355 non-null  object
 2   fecha              17355 non-null  object
 3   requested_test_kv  17355 non-null  object
 4   test_kv            17355 non-null  object
 5   ma                 17355 non-null  object
 6   watts              17355 non-null  object
 7   measured_cap       17355 non-null  object
 8   pfm                17355 non-null  object
 9   pfm_tipup          17355 non-null  object
 10  group_number       17355 non-null  object
 11  stator_kv          17355 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [7]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [10]:
#Seleccionar columnas de interés para primera parte del proceso
columnas_obj = ['serial', 'fecha' , 'stator_kv','mfr', 'group_number', 'requested_test_kv', 'test_kv', 'ma', 'watts',
                'measured_cap', 'pfm']

# Crea un nuevo DataFrame con las columnas seleccionadas
df = df[columnas_obj].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17355 entries, 0 to 17354
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   serial             17355 non-null  object
 1   fecha              17355 non-null  object
 2   stator_kv          17355 non-null  object
 3   mfr                17355 non-null  object
 4   group_number       17355 non-null  object
 5   requested_test_kv  17355 non-null  object
 6   test_kv            17355 non-null  object
 7   ma                 17355 non-null  object
 8   watts              17355 non-null  object
 9   measured_cap       17355 non-null  object
 10  pfm                17355 non-null  object
dtypes: object(11)
memory usage: 1.5+ MB


In [11]:
import pandas as pd

# Suponiendo que 'df' es tu DataFrame
# Seleccionar desde la segunda fila hasta el final
df = df.iloc[1:]

# Mostrar el DataFrame resultante
print(df)

         serial                fecha  ...       measured_cap                 pfm
1        715015  2018-11-27 16:07:37  ...  948070.8058617892   1.086489900421389
2        715015  2018-11-27 16:07:37  ...  950296.2541318992  1.2805929864125465
3        715015  2018-11-27 16:07:37  ...  947210.9638863913  1.4505311233765736
4        715015  2018-11-27 16:07:37  ...  954618.2582198525   1.627991251472089
5        715015  2018-11-27 16:07:37  ...  945508.9582388608  1.0811923796509393
...         ...                  ...  ...                ...                 ...
17350  82EU7701  2005-10-01 17:29:11  ...             3318.6                 2.0
17351  82EU7701  2005-10-01 17:29:11  ...             2988.4                1.97
17352  82EU7701  2005-10-01 17:29:11  ...             2994.2                 1.9
17353  82EU7701  2005-10-01 17:29:11  ...             2996.4                1.95
17354  82EU7701  2005-10-01 17:29:11  ...             2998.4                2.01

[17354 rows x 11 columns]


In [12]:
df['mfr']

1        Toshiba
2        Toshiba
3        Toshiba
4        Toshiba
5        Toshiba
          ...   
17350     Alstom
17351     Alstom
17352     Alstom
17353     Alstom
17354     Alstom
Name: mfr, Length: 17354, dtype: object


In [13]:
#Estandarizar valores de columna group_number

# Reemplazar los valores en la columna group_number
df['group_number'] = df['group_number'].replace({'0': 'GST', '1': 'GST', '2': 'GST', '3': 'UST', '4': 'UST', '5': 'UST', '6':'GST'})

#Deteccion de group_number incorrecto
valores_grnuin = df['group_number'].unique()
valores_grnuin

array(['GST', 'UST'], dtype=object)


In [14]:
df['group_number']

1        GST
2        GST
3        GST
4        GST
5        GST
        ... 
17350    UST
17351    UST
17352    UST
17353    UST
17354    UST
Name: group_number, Length: 17354, dtype: object


In [15]:
# Estandarizacion de columna mfr empleando diccionario
# Diccionario de mapeo de valores originales a valores deseados
mapeo = {
  'Ansal' : 'ANS',
  'General Electric' : 'GE',
  'Brown Boveri Company' : 'GMX',
  'Mitsubishi' : 'MIT',
  'ENERGOMEX' : 'ENG',
  'Rade-Koncar' : 'RK',
  'Toshiba' : 'TOS',
  'AEG Power Tool Corp.' : 'GMX',
  'Alstom' : 'ALS',
  'ASEA' : 'GMX',
  'RADE KONCAR' : 'RK',
  'Westinghouse Electric' : 'OTH',
  'TIBB' : 'GMX',
  'ABB (ASEA-Brown Boveri)' : 'GMX',
  'Koch & Sterzel' : 'OTH',
  'Hitachi' : 'HIT',
  'ASEA Inc.' : 'GMX',
  'Cenemesa' : 'OTH',
  'HITACHI' : 'HIT',
  'Raychem' : 'OTH',
  'Brush Ltd. (H-S Group)' : 'OTH',
  'HARBIN ELECTRIC MACHINERY' : 'OTH',
  'Magnetek' : 'OTH',
  'GAMESA' : 'OTH'
}

# Cambiar los valores en la columna 'mfr' utilizando el diccionario
df['mfr'] = df['mfr'].map(mapeo)
df['mfr']

1        TOS
2        TOS
3        TOS
4        TOS
5        TOS
        ... 
17350    ALS
17351    ALS
17352    ALS
17353    ALS
17354    ALS
Name: mfr, Length: 17354, dtype: object


In [16]:
# Contar la cantidad de ceros en la columna
cantidad_ceros = (df['pfm'] == '0.0').sum()

# Contar la cantidad de NaN en la columna
cantidad_nan = (df['pfm'] == '').sum()

print(f"Cantidad de ceros en {'pfm'}: {cantidad_ceros}")
print(f"Cantidad de NaN en {'pfm'}: {cantidad_nan}")
df.info()

Cantidad de ceros en pfm: 43
Cantidad de NaN en pfm: 1363
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17354 entries, 1 to 17354
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   serial             17354 non-null  object
 1   fecha              17354 non-null  object
 2   stator_kv          17354 non-null  object
 3   mfr                17354 non-null  object
 4   group_number       17354 non-null  object
 5   requested_test_kv  17354 non-null  object
 6   test_kv            17354 non-null  object
 7   ma                 17354 non-null  object
 8   watts              17354 non-null  object
 9   measured_cap       17354 non-null  object
 10  pfm                17354 non-null  object
dtypes: object(11)
memory usage: 1.5+ MB


In [17]:
#Eliminar filas con 'pfm'= 0 o NaN
columna_filtrar = 'pfm'
df = df[(df[columna_filtrar] != '0.0') & (df[columna_filtrar] != '')]

# Contar la cantidad de ceros en la columna
cantidad_ceros = (df['pfm'] == '0.0').sum()

# Contar la cantidad de NaN en la columna
cantidad_nan = (df['pfm'] == '').sum()

print(f"Cantidad de ceros en {'pfm'}: {cantidad_ceros}")
print(f"Cantidad de NaN en {'pfm'}: {cantidad_nan}")
df.info()

Cantidad de ceros en pfm: 0
Cantidad de NaN en pfm: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15948 entries, 1 to 17354
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   serial             15948 non-null  object
 1   fecha              15948 non-null  object
 2   stator_kv          15948 non-null  object
 3   mfr                15948 non-null  object
 4   group_number       15948 non-null  object
 5   requested_test_kv  15948 non-null  object
 6   test_kv            15948 non-null  object
 7   ma                 15948 non-null  object
 8   watts              15948 non-null  object
 9   measured_cap       15948 non-null  object
 10  pfm                15948 non-null  object
dtypes: object(11)
memory usage: 1.5+ MB


In [18]:
df['requested_test_kv']

1        2.0
2        4.0
3        6.0
4        8.0
5        2.0
        ... 
17350    8.0
17351    2.0
17352    4.0
17353    6.0
17354    8.0
Name: requested_test_kv, Length: 15948, dtype: object


In [19]:
#Estandarizacion de valores no válidos para columna requested_text_kv (rtkv)
#conservar sólo valores válidos de tension de prueba

# Valores permitidos para la variable "requested_test_kv"
validos_tkv = ['1.0','2.0','3.0', '4.0', '6.0', '8.0', '10.0']

# Filtrar el DataFrame para conservar solo las filas con los valores permitidos
df = df[df['requested_test_kv'].isin(validos_tkv)]

# Obtener los valores únicos de la columna 'requested_test_kv'
valores_rtkv = df['requested_test_kv'].unique()
valores_rtkv

array(['2.0', '4.0', '6.0', '8.0', '1.0', '3.0', '10.0'], dtype=object)


In [20]:
df.tail()

         serial                fecha stator_kv  ...  watts measured_cap   pfm
17350  82EU7701  2005-10-01 17:29:11      13.8  ...  2.505       3318.6   2.0
17351  82EU7701  2005-10-01 17:29:11      13.8  ...  2.216       2988.4  1.97
17352  82EU7701  2005-10-01 17:29:11      13.8  ...  2.141       2994.2   1.9
17353  82EU7701  2005-10-01 17:29:11      13.8  ...  2.207       2996.4  1.95
17354  82EU7701  2005-10-01 17:29:11      13.8  ...  2.274       2998.4  2.01

[5 rows x 11 columns]


In [21]:

# Suponiendo que 'df' es tu DataFrame
# Filtrar donde 'pfm' es '0.0', 0, o ''
datos_filtrados = df[df['pfm'].isin(['0.0', 0]) | (df['pfm'] == '')]

# Mostrar los datos filtrados
print(datos_filtrados)

Empty DataFrame
Columns: [serial, fecha, stator_kv, mfr, group_number, requested_test_kv, test_kv, ma, watts, measured_cap, pfm]
Index: []


In [25]:
from pyspark.sql import SparkSession
from awsglue.dynamicframe import DynamicFrame

# Inicializa la sesión de Spark
spark = SparkSession.builder.appName("pandasToSpark").getOrCreate()

# Suponiendo que 'pandas_df' es tu DataFrame de pandas
spark_df = spark.createDataFrame(df)

# Convertir el DataFrame de Spark en un DynamicFrame
glueContext = GlueContext(spark.sparkContext)
dyf = DynamicFrame.fromDF(spark_df, glueContext, "dyf")

In [26]:
dyf.printSchema()


root
|-- serial: string
|-- fecha: string
|-- stator_kv: string
|-- mfr: string
|-- group_number: string
|-- requested_test_kv: string
|-- test_kv: string
|-- ma: string
|-- watts: string
|-- measured_cap: string
|-- pfm: string


In [28]:
# Suponiendo que 'dyf' es tu DynamicFrame
num_registros = dyf.count()

# Imprimir el número de registros
print("Número de registros en el DynamicFrame:", num_registros)

Número de registros en el DynamicFrame: 15604


In [27]:
# Ahora puedes usar el dyf con el código para escribir en S3
s3output_parquet = glueContext.getSink(
    path="s3://proyectointegrador20232/trusted/all_files_E_parquet/",
    connection_type="s3",
    updateBehavior="UPDATE_IN_DATABASE",
    partitionKeys=[],
    compression="snappy",
    enableUpdateCatalog=True,
    transformation_ctx="s3output_parquet",
)
s3output_parquet.setCatalogInfo(
    catalogDatabase="proyectoint20232", catalogTableName="file_csv",
)
s3output_parquet.setFormat("glueparquet")
s3output_parquet.writeFrame(dyf)